<a href="https://colab.research.google.com/github/asakata0821/Aisa_INFO5731_-Fall2023/blob/main/Sakata_Exercise_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fourth in-class-exercise (40 points in total, 03/28/2022)**

Question description: Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks:

## (1) (10 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [73]:
import nltk
from nltk.corpus import stopwords
import requests
from bs4 import BeautifulSoup
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
from gensim.models import CoherenceModel
import numpy as np
from pprint import pprint

tokenizer = RegexpTokenizer(r'\w+')
nltk.download('stopwords')
stop_words = stopwords.words('english')
p_stemmer = PorterStemmer()

#Collecting text data (This is the same data as the last class exercise assignment)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

url = 'https://www.jamsj.org/manabu/aiko-ebihara'

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
elements = soup.find_all(class_='sqs-block html-block sqs-block-html', id='block-yui_3_17_2_1_1587148327627_5945')

texts = []

for element in elements:
    paragraphs = element.find_all('p')
    for p in paragraphs:
        text = p.get_text()
        texts.append(text)


#Because Dictionary function works on a element in dictionaries, not a string
stemmed_sentences = []
#Splitting sentences into words
for sentence in texts:
  sentence = sentence.lower()
  words = tokenizer.tokenize(sentence)

  #Removing stopwords from each sentences
  stopword_removed_words = [word for word in words if not word in stop_words]
  stemmed_words = [p_stemmer.stem(word) for word in stopword_removed_words]
  stemmed_sentences.append(stemmed_words)

#Calculating vectors for each words
dictionary = corpora.Dictionary(stemmed_sentences)
corpus = [dictionary.doc2bow(item) for item in stemmed_sentences]

#Finding the best coherence score
coherence_scores_LDA = []
k_values = range(2, 11)
for k in k_values:
  #Applying LDA model
  ldamodel = models.ldamodel.LdaModel(corpus, num_topics=k, id2word = dictionary, passes=20)
  #pprint(ldamodel.print_topics(num_topics=k, num_words=5))

  # Compute Coherence Score
  coherence_model_lda = CoherenceModel(model=ldamodel, texts=stemmed_sentences, dictionary=dictionary, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  coherence_scores_LDA.append(coherence_lda)
  #print('\nCoherence Score: ', coherence_lda)
print(coherence_scores_LDA)

#Highest value in the coherence_scores list with its kvalue and topics
best_k = k_values[np.argmax(coherence_scores_LDA)]
print('K Value:', best_k)
print('Highest Coherence Score:', max(coherence_scores_LDA))
pprint(ldamodel.print_topics(num_topics=best_k, num_words=5))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[0.6814636126475209, 0.4458241886263039, 0.7426871804898896, 0.5780847169741866, 0.7084188506282851, 0.7773973160103654, 0.8245161423984153, 0.7054675015155988, 0.8851941990649124]
K Value: 10
Highest Coherence Score: 0.8851941990649124
[(0,
  '0.006*"journal" + 0.006*"friend" + 0.006*"kept" + 0.006*"mantl" + '
  '0.006*"hirasaki"'),
 (1,
  '0.006*"aiko" + 0.006*"velora" + 0.006*"japanes" + 0.006*"nation" + '
  '0.006*"hirasaki"'),
 (2,
  '0.048*"aiko" + 0.033*"william" + 0.017*"year" + 0.017*"famili" + '
  '0.017*"tanaka"'),
 (3,
  '0.070*"love" + 0.036*"took" + 0.036*"tanaka" + 0.019*"seem" + 0.019*"unit"'),
 (4,
  '0.052*"aiko" + 0.035*"velora" + 0.035*"nation" + 0.035*"japanes" + '
  '0.035*"american"'),
 (5,
  '0.006*"aiko" + 0.006*"william" + 0.006*"carri" + 0.006*"would" + '
  '0.006*"stand"'),
 (6,
  '0.007*"aiko" + 0.006*"velora" + 0.006*"american" + 0.006*"japanes" + '
  '0.006*"nation"'),
 (7,
  '0.006*"aiko" + 0.006*"arrang" + 0.006*"william" + 0.006*"live" + '
  '0.006*"re

In [74]:
!pip install pyLDAvis

In [77]:
#For running pyLDAvis, I needed to downgrade pandas version
!pip install pandas==1.5.3

  Using cached pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.2
    Uninstalling pandas-2.1.2:
      Successfully uninstalled pandas-2.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
pyldavis 3.4.1 requires pandas>=2.0.0, but you have pandas 1.5.3 which is incompatible.


In [78]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)

pyLDAvis.display(LDAvis_prepared)


## (2) (10 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [79]:
import os.path
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import numpy as np
import matplotlib.pyplot as plt

#Using the same stopword removed text data from question(1)
#'stemmed_sentences'

#Using the same corpus from question (1)
#'corpus'

#Similarly to LDA model, finding the highest coherence score and its k-value
coherence_scores_LSA = []
k_values = range(2, 11)
for k in k_values:
  #Applying LSA model
  lsamodel = LsiModel(corpus, num_topics=k, id2word = dictionary)
  #pprint(ldamodel.print_topics(num_topics=k, num_words=5))

  # Compute Coherence Score
  coherence_model_lsa = CoherenceModel(model=lsamodel, texts=stemmed_sentences, dictionary=dictionary, coherence='c_v')
  coherence_lsa = coherence_model_lsa.get_coherence()
  coherence_scores_LSA.append(coherence_lsa)
  #print('\nCoherence Score: ', coherence_lsa)
print(coherence_scores_LSA)

#Highest value in the coherence_scores list with the k-value and topics
best_k = k_values[np.argmax(coherence_scores_LSA)]
print('K Value:', best_k)
print('Highest Coherence Score:', max(coherence_scores_LSA))
pprint(lsamodel.print_topics(num_topics=best_k, num_words=5))





[0.7879033629838371, 0.8166600980584414, 0.685819263736162, 0.5927091624768042, 0.5927091624768042, 0.5927091624768042, 0.7008740001173767, 0.7008740001173767, 0.685819263736162]
K Value: 3
Highest Coherence Score: 0.8166600980584414
[(0,
  '0.583*"aiko" + 0.288*"william" + 0.187*"famili" + 0.187*"year" + '
  '0.147*"velora"'),
 (1,
  '0.500*"love" + 0.240*"tanaka" + 0.236*"took" + -0.191*"aiko" + '
  '0.146*"permeat"'),
 (2,
  '0.276*"japanes" + 0.248*"american" + 0.248*"nation" + 0.216*"velora" + '
  '0.149*"photo"')]


## (3) (10 points) Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [58]:
pip install lda2vec

In [72]:
#As I sent an email, I cannot define the LDA2Vec model.
#I tried importing multiple packages in different names, but still it shows there's no such LDA2Vec function
#,which is used to define the model.

import nltk
from nltk.corpus import stopwords
import requests
from bs4 import BeautifulSoup
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
from gensim.models import CoherenceModel
import numpy as np
from pprint import pprint
from lda2vec import LDA2Vec

tokenizer = RegexpTokenizer(r'\w+')
nltk.download('stopwords')
stop_words = stopwords.words('english')
p_stemmer = PorterStemmer()

topic_coherence = stemmed_sentences.topic_coherence


# Finding the best coherence score
coherence_scores_LDA2vec = []
k_values = range(2, 11)
for k in k_values:
    # Applying LDA2vec model
    lda2vec_model = LDA2Vec(corpus=corpus, num_topics=k, vector_size=100, window=5, min_count=1)
    # pprint(lda2vec_model.print_topics(num_topics=k, num_words=5))

    # Compute Coherence Score
    coherence_model_lda2vec = CoherenceModel(model=lda2vec_model, texts=stemmed_sentences, dictionary=dictionary, coherence='c_v')
    coherence_lda2vec = coherence_model_lda2vec.get_coherence()
    coherence_scores_LDA2vec.append(coherence_lda2vec)
    # print('\nCoherence Score: ', coherence_lda2vec)
print(coherence_scores_LDA2vec)

# Highest value in the coherence_scores list with its kvalue and topics
best_k = k_values[np.argmax(coherence_scores_LDA2vec)]
print('K Value:', best_k)
print('Highest Coherence Score:', max(coherence_scores_LDA2vec))
pprint(lda2vec_model.print_topics(num_topics=best_k, num_words=5))


ImportError: ignored

In [34]:
#This is used the other website as a reference.
#It shows some outputs but I don't think this is LDA2Vec.

import nltk
from nltk.corpus import stopwords
import requests
from bs4 import BeautifulSoup
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
from gensim.models import CoherenceModel
import numpy as np
from pprint import pprint
import lda2vec


#Using the same stopword removed text data from question(1)
#'stemmed_sentences'

#Using the same corpus from question (1)
#'corpus'

# Finding the best coherence score
top_n = 10
topic_to_topwords = {}
for j, topic_to_word in enumerate(stemmed_sentences):
    top = np.argsort(topic_to_word)[::-1][:top_n]
    msg = 'Topic %i '  % j

    # Check if the index is within the valid range
    top_words = [sentence.strip() for i in top if i < len(stemmed_sentences) for sentence in stemmed_sentences[i]]

    msg += ' '.join(top_words)
    print(msg)
    topic_to_topwords[j] = top_words




Topic 0 
Topic 1 stori aiko ebihara realli begin friendship forg two famili live salem oregon prior start wwii aiko parent maki frank restaur owner full time cook tokio sukiyaki live restaur cramp bedroom three young children aiko way simpli room newborn babi arrang made famili town william care year arrang work well velora child edward alic william would come embrac aiko proud aiko want claim littl sister wrote three year william rais aiko home frequent visit check in tanaka
Topic 2 stori aiko ebihara realli begin friendship forg two famili live salem oregon prior start wwii aiko parent maki frank restaur owner full time cook tokio sukiyaki live restaur cramp bedroom three young children aiko way simpli room newborn babi arrang made famili town william care year arrang work well velora child edward alic william would come embrac aiko proud aiko want claim littl sister wrote three year william rais aiko home frequent visit check in tanaka
Topic 3 


## (4) (10 points) Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [80]:
pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 45.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.6 MB/s eta 0:00:00
  Preparing metadata (

In [81]:
#For the rest of the questions, I used the same textdata (1 list with 4 strings) as the last class exercise assignment.
#But, for using KeyBERT it seems I need more data.
#So I collected more data and now I obtain 73 textdata for this question.

import requests
from bs4 import BeautifulSoup

all_list = []

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

url = 'https://www.jamsj.org/manabu'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
result = soup.find_all('div', class_='margin-wrapper')

# Collecting names, descriptions, and transcripts
for item in result:
    narrator_page_link_parts = item.find('a', class_='image-slide-anchor')
    if narrator_page_link_parts:
        narrator_page_link = narrator_page_link_parts.get('href')
        narrator_page_url = 'https://www.jamsj.org' + narrator_page_link

        narrator_response = requests.get(narrator_page_url, headers=headers)
        narrator_soup = BeautifulSoup(narrator_response.content, 'html.parser')   # accessing each narrator's page
        name_elements = narrator_soup.find('h1', style='text-align:center;white-space:pre-wrap;')
        text_elements = narrator_soup.find_all('p', style='white-space:pre-wrap;')

        #if name_elements not in duplicates:
        name = name_elements.get_text(strip=True)

        #else:
            #continue

        #if name in duplicates:
            #continue

        transcript_started = False
        transcript = []
        description = []
        strong_text = ""

        for p in text_elements:
            if p.find('strong'):
                # Text under <strong> tag is considered questions by interviewers
                strong_text = 'Interviewer: ' + p.find('strong').get_text(strip=True)
                transcript_started = True
                continue

            if transcript_started:
                # Content after <strong> tag is considered an answer
                answer = 'Narrator: ' + p.get_text(strip=True).replace('\n','')
                  # Remove newline characters
                transcript.append(strong_text)
                transcript.append(answer)
                strong_text = ""
            else:
                # Content before <strong> tag is description
                description.append(p.get_text(strip=True))

        # Append name, description, and transcript to all_list
        all_list.append({
            'Name': name,
            'Description': description,
            'Transcript': transcript
        })



In [82]:
import pandas as pd
from bertopic import BERTopic
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

stop_words = stopwords.words('english')

df = pd.DataFrame(all_list)
descriptions = df['Description']

stemmed_descriptions = []
for description in descriptions:
  for line in description:
    line = line.lower()
    #print(line)
    words = tokenizer.tokenize(line)

        #Removing stopwords from each sentences
    stopword_removed_words = [word for word in words if not word in stop_words]
    stemmed_words = [p_stemmer.stem(word) for word in stopword_removed_words]
    stemmed_descriptions.append(stemmed_words)

stemmed_descriptions = [str(row) for row in stemmed_descriptions]

topic_model = BERTopic(language="english")
topics, probs = topic_model.fit_transform(stemmed_descriptions)


In [83]:
freq = topic_model.get_topic_info()
freq

,Topic,Count,Name,Representation,Representative_Docs
0,0,210,0_famili_japanes_camp_year,"[famili, japanes, camp, year, would, american,...","[['alreadi', '21', 'time', 'pearl', 'harbor', ..."
1,1,16,1_born_ca_name_june,"[born, ca, name, june, august, california, 193...","[['mari', 'shizuko', 'second', 'eldest', 'daug..."


In [84]:
topic_model.get_topic(0)

[('famili', 0.04834439121631059),
 ('japanes', 0.04181624080356355),
 ('camp', 0.0415344085046661),
 ('year', 0.04011235153469671),
 ('would', 0.036606286829578234),
 ('american', 0.03357398698765767),
 ('war', 0.03138447791415682),
 ('father', 0.027818052745663575),
 ('live', 0.027485491677116138),
 ('one', 0.026815892337008998)]

In [88]:
topic_model.visualize_topics(topics)

In [89]:
topic_model.visualize_distribution(probs, min_probability=0.015)

In [93]:
topic_model.visualize_hierarchy(top_n_topics=len(topics))

In [94]:
topic_model.visualize_barchart(top_n_topics=len(topics))

## (5) (10 extra points) Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.

In [ ]:
"""
I think BERTopic is the best method to create topic categories and classify text data into those categories.
It also shows the importance scores by each keyword and more than anything,
the variety of visualizations in the same BERT package is very enriching.
These (above) are some examples, and there are more visualizations I can run and create.
That is very helpful to analyze from different points of view and enables us to deeply understand
its topics and relationships.

The other two methods (because I couldn't get outputs using the LDA2Vec model) are also useful.
However, if I think of things that I can do with the package only, BERTopic is better.
"""